In [1]:
!pwd
import sys

%reload_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext lab_black
sys.executable

/c/Users/81908/jupyter_notebook/pytorch_lightning_work/kaggle_Cassava/notebook/check_oof/check_pkl


'C:\\Users\\81908\\Anaconda3\\envs\\lightning\\python.exe'

In [2]:
import json
import pandas as pd

ROOT_DIR = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\input\cassava-leaf-disease-classification"

with open(f"{ROOT_DIR}/label_num_to_disease_map.json", "r") as f:
    name_mapping = json.load(f)
name_mapping = {int(k): v for k, v in name_mapping.items()}

df = pd.read_csv(f"{ROOT_DIR}/train.csv")
df

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3
...,...,...
21392,999068805.jpg,3
21393,999329392.jpg,3
21394,999474432.jpg,1
21395,999616605.jpg,4


In [3]:
import sys

sys.path.append(r"C:\Users\81908\MyGitHub\kaggle_Cassava\code")

from params import base_data

noise_image_id = base_data.noise_image_id
noise_idx = list(df[df["image_id"].isin(noise_image_id)].index)

df["noize"] = 0
df.loc[noise_idx, "noize"] = 1
df["noize"] = df["noize"].astype(int)
df

,image_id,label,noize
0,1000015157.jpg,0,0
1,1000201771.jpg,3,0
2,100042118.jpg,1,1
3,1000723321.jpg,1,0
4,1000812911.jpg,3,0
...,...,...,...
21392,999068805.jpg,3,0
21393,999329392.jpg,3,0
21394,999474432.jpg,1,0
21395,999616605.jpg,4,0


In [4]:
import glob
import pathlib
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn

# oof_list = []
# cfms = []
# ens_weights = []
# model_names = []
# labels = []
#
#
# def check_oof(Y, pkl_path=""):
#    name = str(pathlib.Path(pkl_path).name)
#    Y_oof = pickle.load(open(f"{pkl_path}", "rb"))
#    oof = accuracy_score(Y, Y_oof.values.argmax(1))
#    cfm = confusion_matrix(Y, Y_oof.values.argmax(1))
#    label = Y_oof.values.argmax(1)
#
#    print(f"{name} oof:    \t{round(oof, 4)}")
#
#    oof_list.append(Y_oof.values)
#    cfms.append(cfm)
#    ens_weights.append(oof)
#    model_names.append(name)
#    labels.append(label)
#
#    return Y_oof.values, oof
#
#
## --------- anonamename ---------
# print("-" * 100)
ano_m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\check_oof\cassava-emsemble-v2_tta_oof\kaggle_upload_oof_tta"
# for pkl_path in sorted(glob.glob(f"{ano_m_dir}/*.pkl")):
#    if "seresnext50_32x4d_fmix_tta.pkl" != pathlib.Path(pkl_path).name:
#        Y_oof, oof = check_oof(df["label"].values, pkl_path=pkl_path)
#
## --------- SiNpcw ---------
# print("-" * 100)
sin_m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassavapkl"
# for pkl_path in sorted(glob.glob(f"{sin_m_dir}/*.pkl")):
#    if (
#        "ex12" in pathlib.Path(pkl_path).name
#        or "ex15" in pathlib.Path(pkl_path).name
#        or "ex16" in pathlib.Path(pkl_path).name
#        or "ex02" in pathlib.Path(pkl_path).name
#        or "ex09" in pathlib.Path(pkl_path).name
#        or "ex10" in pathlib.Path(pkl_path).name
#        or "ex05" in pathlib.Path(pkl_path).name
#        or "22019640" in pathlib.Path(pkl_path).name
#        or "22019725" in pathlib.Path(pkl_path).name
#        or "22019613" in pathlib.Path(pkl_path).name
#        or "22019720" in pathlib.Path(pkl_path).name
#    ):
#        Y_oof, oof = check_oof(df["label"].values, pkl_path=pkl_path)

In [5]:
def ens_weight_pkl(pkls, ws):
    ens = np.zeros((df.shape[0], 5))
    for pkl, w in zip(pkls, ws):
        pre = pickle.load(open(f"{pkl}", "rb")).values
        ens += (w / sum(ws)) * pre
    ens_df = pd.DataFrame(ens, columns=name_mapping.values())
    ens_df["pred_label"] = ens.argmax(1)
    print("acc:", accuracy_score(df["label"].values, ens.argmax(1)))
    # display(ens_df)
    return ens.argmax(1)


def ens_cfm_pkl(pkls, cfm_ws):
    ens = np.zeros((df.shape[0], 5))
    for i, pkl in enumerate(pkls):
        pre = pickle.load(open(f"{pkl}", "rb")).values
        ens += cfm_ws[i] * pre
    ens_df = pd.DataFrame(ens, columns=name_mapping.values())
    ens_df["pred_label"] = ens.argmax(1)
    print("acc:", accuracy_score(df["label"].values, ens.argmax(1)))
    # display(ens_df)
    return ens.argmax(1)

# v20

In [6]:
# 5model, oof=0.906388
pkls_v20_5m = [
    f"{ano_m_dir}/resnest101e_cleanlab_noise_cutmix_tta.pkl",
    f"{ano_m_dir}/tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over_tta.pkl",
    f"{sin_m_dir}/22019725_tta3_efficientnet-b4.pkl",
    f"{sin_m_dir}/ex02C_tta5_efficientnet-b4.pkl",
    f"{sin_m_dir}/ex10C_tta5_timm-resnest101e.pkl",
]
ens_weights_v20_5m = [
    0.8948918072627003,
    0.8881151563303267,
    0.8947983362153573,
    0.9008272187689863,
    0.9005935411506286,
]
df["v20_5m"] = ens_weight_pkl(pkls_v20_5m, ens_weights_v20_5m)

acc: 0.9063887460858999


In [7]:
# 6model, oof=0.906715
pkls_v20_6m = [
    f"{ano_m_dir}/resnest101e_cleanlab_noise_cutmix_tta.pkl",
    f"{ano_m_dir}/resnest101e_tta.pkl",
    f"{ano_m_dir}/tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over_tta.pkl",
    f"{sin_m_dir}/ex02C_tta5_efficientnet-b4.pkl",
    f"{sin_m_dir}/ex09C_tta3_efficientnet-b4.pkl",
    f"{sin_m_dir}/ex10C_timm-resnest101e.pkl",
]
ens_weights_v20_6m = [
    0.8948918072627003,
    0.8941907744076273,
    0.8881151563303267,
    0.9008272187689863,
    0.9005000701032855,
    0.9011076319110156,
]
df["v20_6m"] = pred_label = ens_weight_pkl(pkls_v20_6m, ens_weights_v20_6m)

acc: 0.9067158947516006


In [8]:
# 5model, cfm_oof=0.9064354
pkls_v20_5m_cfm = [
    f"{ano_m_dir}/resnest101e_cleanlab_noise_cutmix_tta.pkl",
    f"{ano_m_dir}/tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over_tta.pkl",
    f"{sin_m_dir}/22019725_tta3_efficientnet-b4.pkl",
    f"{sin_m_dir}/ex02C_tta5_efficientnet-b4.pkl",
    f"{sin_m_dir}/ex10C_tta3_timm-resnest101e.pkl",
]
ens_weights_v20_5m_cfm = [
    [0.18955142, 0.19892111, 0.2004905, 0.19968324, 0.20433532],
    [0.21772429, 0.19487525, 0.20161455, 0.19770742, 0.19437025],
    [0.19310722, 0.20128119, 0.1966074, 0.20107886, 0.19539758],
    [0.20650985, 0.20150596, 0.20232986, 0.20027912, 0.20289706],
    [0.19310722, 0.2034165, 0.19895769, 0.20125135, 0.20299979],
]
df["v20_5m_cfm"] = ens_cfm_pkl(pkls_v20_5m_cfm, ens_weights_v20_5m_cfm)

acc: 0.9064354816095714


In [9]:
# 6model, cfm_oof=0.906528
# ここには書いてないが v19もこの組み合わせがbest
pkls_v20_6m_cfm = [
    f"{ano_m_dir}/deit_base_patch16_224_tta.pkl",
    f"{ano_m_dir}/resnest101e_tta.pkl",
    f"{ano_m_dir}/tf_efficientnet_b4_ns_tta.pkl",
    f"{sin_m_dir}/22019640_timm-resnest200e.pkl",
    f"{sin_m_dir}/22019720_efficientnet-b4.pkl",
    f"{sin_m_dir}/ex02C_tta3_efficientnet-b4.pkl",
]
ens_weights_v20_6m_cfm = [
    [0.15281933, 0.16220547, 0.15501994, 0.16721397, 0.1671266],
    [0.17330265, 0.16399623, 0.16585746, 0.1663018, 0.16945498],
    [0.17146145, 0.16493874, 0.16941217, 0.16649726, 0.16117627],
    [0.16593786, 0.17229029, 0.17071268, 0.16660151, 0.16410831],
    [0.16409666, 0.16804901, 0.16837177, 0.16695335, 0.1680752],
    [0.17238205, 0.16852026, 0.17062598, 0.16643211, 0.17005864],
]
df["v20_6m_cfm"] = ens_cfm_pkl(pkls_v20_6m_cfm, ens_weights_v20_6m_cfm)

acc: 0.9065289526569145


# v23 
- 相関係数<0.95

In [10]:
# 5model, oof=0.906435
pkls_v23_5m = [
    f"{ano_m_dir}/resnest101e_cleanlab_noise_cutmix_tta.pkl",
    f"{ano_m_dir}/tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over_tta.pkl",
    f"{sin_m_dir}/ex02C_tta5_efficientnet-b4.pkl",
    f"{sin_m_dir}/ex10C_timm-resnest101e.pkl",
    f"{sin_m_dir}/ex12C_tta3_timm-resnest101e.pkl",
]
ens_weights_v23_5m = [
    0.8948918072627003,
    0.8881151563303267,
    0.9008272187689863,
    0.9011076319110156,
    0.9008272187689863,
]
df["v23_5m"] = ens_weight_pkl(pkls_v23_5m, ens_weights_v23_5m)

acc: 0.9064354816095714


In [11]:
# 6model, oof=0.906669
pkls_v23_6m = [
    f"{ano_m_dir}/deit_base_patch16_224_tta.pkl",
    f"{ano_m_dir}/resnest101e_tta.pkl",
    f"{ano_m_dir}/tf_efficientnet_b4_ns_tta.pkl",
    f"{sin_m_dir}/22019640_timm-resnest200e.pkl",
    f"{sin_m_dir}/22019720_efficientnet-b4.pkl",
    f"{sin_m_dir}/ex02C_tta3_efficientnet-b4.pkl",
]
ens_weights_v23_6m = [
    0.8853110249100341,
    0.8941907744076273,
    0.8924148245081086,
    0.8976024676356499,
    0.897275318969949,
    0.8996120951535262,
]
df["v23_6m"] = ens_weight_pkl(pkls_v23_6m, ens_weights_v23_6m)

acc: 0.9066691592279291


In [12]:
# 5model cfm_oof=0.9064354
# v20 5model, cfm_oof=0.9064354 と同じ組み合わせ
pkls_v23_5m_cfm = pkls_v20_5m_cfm
ens_weights_v23_5m_cfm = ens_weights_v20_5m_cfm
# pred_label = ens_cfm_pkl(pkls_v23_5m_cfm, ens_weights_v23_5m_cfm)

In [13]:
# 6model, cfm_oof=

# v24
- 相関係数<0.95

In [14]:
# 5model, oof=0.90643548
# v23 5model, oof=0.906435 と同じ組み合わせ
pkls_v24_5m = pkls_v23_5m
ens_weights_v24_5m = ens_weights_v23_5m
# pred_label = ens_weight_pkl(pkls_v24_5m, ens_weights_v24_5m)

In [15]:
# 6model, oof=0.906528
pkls_v24_6m = [
    f"{ano_m_dir}/deit_base_patch16_224_tta.pkl",
    f"{ano_m_dir}/resnest101e_cleanlab_noise_cutmix_tta.pkl",
    f"{ano_m_dir}/tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over_tta.pkl",
    f"{sin_m_dir}/ex02C_tta5_efficientnet-b4.pkl",
    f"{sin_m_dir}/ex10C_tta5_timm-resnest101e.pkl",
    f"{sin_m_dir}/ex16C_timm-resnest101e.pkl",
]
ens_weights_v24_6m = [
    0.8853110249100341,
    0.8948918072627003,
    0.8881151563303267,
    0.9008272187689863,
    0.9005935411506286,
    0.9018554002897603,
]
df["v24_6m"] = ens_weight_pkl(pkls_v24_6m, ens_weights_v24_6m)

acc: 0.9065289526569145


In [16]:
# 5model cfm_oof=0.9064354
# v20 5model, cfm_oof=0.9064354 と同じ組み合わせ
pkls_v24_5m_cfm = pkls_v20_5m_cfm
ens_weights_v24_5m_cfm = ens_weights_v20_5m_cfm
# pred_label = ens_cfm_pkl(pkls_v24_5m_cfm, ens_weights_v24_5m_cfm)

In [17]:
# 6model, cfm_oof=0.906575
pkls_v24_6m_cfm = [
    f"{ano_m_dir}/resnest101e_cleanlab_noise_cutmix_tta.pkl",
    f"{ano_m_dir}/resnest101e_tta.pkl",
    f"{ano_m_dir}/tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over_tta.pkl",
    f"{sin_m_dir}/ex02C_tta5_efficientnet-b4.pkl",
    f"{sin_m_dir}/ex10C_timm-resnest101e.pkl",
    f"{sin_m_dir}/ex16C_tta5_timm-resnest101e.pkl",
]
ens_weights_v24_6m_cfm = [
    [0.15615142, 0.16615038, 0.16709249, 0.1664771, 0.16881684],
    [0.16967102, 0.16333427, 0.16291943, 0.16684316, 0.16677983],
    [0.17936007, 0.16277105, 0.1680293, 0.16482985, 0.16058394],
    [0.17012168, 0.1683094, 0.16862545, 0.16697389, 0.16762859],
    [0.15998197, 0.1701868, 0.16607052, 0.16771908, 0.16796809],
    [0.16471384, 0.1692481, 0.16726282, 0.16715692, 0.16822271],
]
df["v24_6m_cfm"] = ens_cfm_pkl(pkls_v24_6m_cfm, ens_weights_v24_6m_cfm)

acc: 0.9065756881805861


# Fukuda optuna best

In [18]:
pkls = [
    f"{ano_m_dir}/tf_efficientnet_b4_ns_tta.pkl",
    f"{ano_m_dir}/resnest101e_cleanlab_noise_cutmix_tta.pkl",
    f"{ano_m_dir}/vit_base_patch32_384.fit_tta.pkl",
    f"{sin_m_dir}/22019632_timm-resnest101e.pkl",
    f"{sin_m_dir}/ex07C_efficientnet-b4.pkl",
    f"{sin_m_dir}/ex16C_timm-resnest101e.pkl",
]
ens_weights = [
    [
        0.18281808105110137,
        0.18281808105110137,
        0.18281808105110137,
        0.18281808105110137,
        0.18281808105110137,
    ],  # tf_efficientnet_b4_ns_tta(0.892415)
    [
        0.2192021079734161,
        0.2192021079734161,
        0.2192021079734161,
        0.2192021079734161,
        0.2192021079734161,
    ],  # resnest101e_cleanlab_noise_cutmix_tta(0.894892)
    [
        0.21204554330471248,
        0.21204554330471248,
        0.21204554330471248,
        0.21204554330471248,
        0.21204554330471248,
    ],  # vit_base_patch32_384.fit_tta(0.883068)
    [
        0.17584641848557123,
        0.17584641848557123,
        0.17584641848557123,
        0.17584641848557123,
        0.17584641848557123,
    ],  # 22019632_timm-resnest101e(0.897275)
    [
        0.23929126514173407,
        0.23929126514173407,
        0.23929126514173407,
        0.23929126514173407,
        0.23929126514173407,
    ],  # ex07C_efficientnet-b4(0.900079)
    [
        0.22521530083249244,
        0.22521530083249244,
        0.22521530083249244,
        0.22521530083249244,
        0.22521530083249244,
    ],  # ex16C_timm-resnest101e(0.901855)
]
df["optuna_best"] = ens_cfm_pkl(pkls, ens_weights)

acc: 0.9067626302752723


In [19]:
pkls = [
    f"{ano_m_dir}/tf_efficientnet_b4_ns_tta.pkl",
    f"{ano_m_dir}/resnest101e_cleanlab_noise_cutmix_tta.pkl",
    f"{ano_m_dir}/vit_base_patch32_384.fit_tta.pkl",
    f"{sin_m_dir}/22019632_timm-resnest101e.pkl",
    f"{sin_m_dir}/ex07C_efficientnet-b4.pkl",
    f"{sin_m_dir}/ex16C_timm-resnest101e.pkl",
]
ens_weights = [
    [
        0.2,
        0.2,
        0.2,
        0.2,
        0.2,
    ],  # tf_efficientnet_b4_ns_tta(0.892415)
    [
        0.2,
        0.2,
        0.2,
        0.2,
        0.2,
    ],  # resnest101e_cleanlab_noise_cutmix_tta(0.894892)
    [
        0.2,
        0.2,
        0.2,
        0.2,
        0.2,
    ],  # vit_base_patch32_384.fit_tta(0.883068)
    [
        0.2,
        0.2,
        0.2,
        0.2,
        0.2,
    ],  # 22019632_timm-resnest101e(0.897275)
    [
        0.2,
        0.2,
        0.2,
        0.2,
        0.2,
    ],  # ex07C_efficientnet-b4(0.900079)
    [
        0.2,
        0.2,
        0.2,
        0.2,
        0.2,
    ],  # ex16C_timm-resnest101e(0.901855)
]
df["optuna_mean"] = ens_cfm_pkl(pkls, ens_weights)

acc: 0.9062952750385568


In [20]:
# 25param oof=0.9076
# https://www.kaggle.com/shun2741/cassava-emsemble-team?scriptVersionId=54256668
pkls = [
    f"{ano_m_dir}/tf_efficientnet_b4_ns_tta.pkl",
    f"{ano_m_dir}/vit_base_patch32_384.fit_tta.pkl",
    f"{ano_m_dir}/resnest101e_cleanlab_noise_cutmix_tta.pkl",
    f"{sin_m_dir}/22019632_timm-resnest101e.pkl",
    f"{sin_m_dir}/ex02C_tta3_efficientnet-b4.pkl",
]
ens_weights = np.array(
    [
        [
            0.22355777129650842,
            0.17028420210498382,
            0.1618069864239824,
            0.24428662425621833,
            0.2356008081512508,
        ],  # 'tf_efficientnet_b4_ns_tta'
        [
            0.2428264056451276,
            0.24258390077453387,
            0.18634503317766682,
            0.1542060546614458,
            0.15582260913215681,
        ],  # 'vit_base_patch32_384.fit_tta'
        [
            0.184468676092403,
            0.1933160347453595,
            0.17310139553235498,
            0.2383757993975987,
            0.16716110929165331,
        ],  # 'resnest101e_cleanlab_noise_cutmix_tta'
        [
            0.19333549839457148,
            0.17153849177293262,
            0.15988223950217587,
            0.1639418875099471,
            0.24291320208579997,
        ],  # '22019632_timm-resnest101e'
        [
            0.17760795307313848,
            0.229447901421209,
            0.2467678201511525,
            0.23396755471730904,
            0.17684789662068565,
        ],  # 'ex02C_tta3_efficientnet-b4'
    ]
)
df["optuna_25para"] = ens_cfm_pkl(pkls, ens_weights)

acc: 0.90760386970136


In [21]:
# 30param oof=0.9071365144646446
# https://www.kaggle.com/shun2741/cassava-emsemble-v2-no-cfm-fast-optuna?scriptVersionId=54477800
pkls = [
    f"{ano_m_dir}/tf_efficientnet_b4_ns_tta.pkl",
    f"{ano_m_dir}/resnest101e_cleanlab_noise_cutmix_tta.pkl",
    f"{ano_m_dir}/vit_base_patch32_384.fit_tta.pkl",
    f"{sin_m_dir}/22019632_timm-resnest101e.pkl",
    f"{sin_m_dir}/ex07C_efficientnet-b4.pkl",
    f"{sin_m_dir}/ex16C_timm-resnest101e.pkl",
]
ens_weights = np.array(
    [
        [
            0.15993657310308937,
            0.1667006928137247,
            0.20747206914503333,
            0.2458062497136567,
            0.22184626791162237,
        ],  # tf_efficientnet_b4_ns_tta(0.892415)
        [
            0.1872043064666931,
            0.23898299486416721,
            0.23677061750797562,
            0.21608829981246142,
            0.18681577143555153,
        ],  # resnest101e_cleanlab_noise_cutmix_tta(0.894892)
        [
            0.22997981315411237,
            0.20323536812316026,
            0.16234257923820855,
            0.2444654160836384,
            0.21985182048378163,
        ],  # vit_base_patch32_384.fit_tta(0.883068)
        [
            0.218712544675187,
            0.20283229866292235,
            0.23406428576598062,
            0.21231168021778468,
            0.186755529950474,
        ],  # 22019632_timm-resnest101e(0.897275)
        [
            0.2259446176489109,
            0.22897128010400072,
            0.15589152843744364,
            0.20001258264726185,
            0.23913190096373976,
        ],  # ex07C_efficientnet-b4(0.900079)
        [
            0.24077307656932864,
            0.2073387615904423,
            0.2265018485219022,
            0.16391181722316367,
            0.20174638005995213,
        ],  # ex16C_timm-resnest101e(0.901855)
    ]
)
df["optuna_30para"] = ens_cfm_pkl(pkls, ens_weights)

acc: 0.9071365144646446


# stacking oof

In [22]:
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\stacking_test\20210212_objective10"
pre = pickle.load(open(f"{m_dir}/Y_pred.pkl", "rb")).values
pre_label = pre.argmax(1)
print("acc:", accuracy_score(df["label"].values, pre_label))
df["stacking_20210212_objective10"] = pre_label

acc: 0.9056409777071552


In [23]:
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\stacking_test\20210212_objective4"
pre = pickle.load(open(f"{m_dir}/Y_pred.pkl", "rb")).values
pre_label = pre.argmax(1)
print("acc:", accuracy_score(df["label"].values, pre_label))
df["stacking_20210212_objective4"] = pre_label

acc: 0.9072767210356592


In [24]:
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\stacking_test\rerun_20210217_oof_best1"
pre = pickle.load(open(f"{m_dir}/Y_pred.pkl", "rb")).values
pre_label = pre.argmax(1)
print("acc:", accuracy_score(df["label"].values, pre_label))
df["stacking_20210217_oof_best1"] = pre_label

acc: 0.9065289526569145


In [25]:
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\stacking_test\rerun_20210212_objective6"
pre = pickle.load(open(f"{m_dir}/Y_pred.pkl", "rb")).values
pre_label = pre.argmax(1)
print("acc:", accuracy_score(df["label"].values, pre_label))
df["stacking_rerun_20210212_objective6"] = pre_label

acc: 0.9072767210356592


# Output

In [26]:
df.to_csv("pred_label.csv")
df

,image_id,label,noize,v20_5m,v20_6m,v20_5m_cfm,v20_6m_cfm,v23_5m,v23_6m,v24_6m,v24_6m_cfm,optuna_best,optuna_mean,optuna_25para,optuna_30para,stacking_20210212_objective10,stacking_20210212_objective4,stacking_20210217_oof_best1,stacking_rerun_20210212_objective6
0,1000015157.jpg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1000201771.jpg,3,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2,100042118.jpg,1,1,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
3,1000723321.jpg,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1000812911.jpg,3,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21392,999068805.jpg,3,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
21393,999329392.jpg,3,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
21394,999474432.jpg,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
21395,999616605.jpg,4,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
